# Extracting value from the payment notes

In [90]:
import pandas as pd
import numpy as np
import psycopg2
import pymongo
import json
import datetime
import pickle
import functions as fn
import matplotlib.pyplot as plt 
%matplotlib inline
import emoji
import regex
from nltk import FreqDist

In [2]:
# load the above mentioned pickle
with open('initial_5pct_transactions.pkl', 'rb') as f:
    initial_5pct = pickle.load(f)

In [3]:
# Retrieve information to connect to the database
keys = fn.get_keys("/Users/jjherranzsarrion/.secret/local_info.json")
username = keys['username']
password = keys['password']

# Select your time ranges and add them in variables
train_window_end = '2018-07-28 23:59:59'
test_window_start = '2018-07-29 00:00:00'
test_window_end = '2018-07-29 23:59:59'
previous_day_start = '2018-07-28 00:00:00'

In [4]:
cursor = fn.extracting_cursor(username, password)
q = f"""SELECT *
        FROM payments p;"""
cursor.execute(q)
payments = pd.DataFrame(cursor.fetchall())
payments.columns = [x[0] for x in cursor.description]

In [8]:
notes = payments['note']

In [91]:
def split_count(text):
    "Function that takes in all the notes and returns the emojis used"
    emoji_list = []
    for note in notes:
        data = regex.findall(r'\X', note)
        for word in note:
            if any(char in emoji.UNICODE_EMOJI for char in word):
                emoji_list.append(word)
    return emoji_list

In [92]:
unique_emojis = split_count(notes)

In [93]:
macbeth_freqdist = FreqDist(unique_emojis)
macbeth_freqdist.most_common(50)

[('🍕', 9110),
 ('💸', 7567),
 ('🏼', 6884),
 ('🏻', 6598),
 ('🏠', 6410),
 ('🌮', 5739),
 ('❤', 5468),
 ('♀', 5105),
 ('🍺', 4628),
 ('🍻', 3830),
 ('🍹', 3493),
 ('⛽', 3348),
 ('🎉', 3291),
 ('🚗', 3115),
 ('☕', 3059),
 ('🍷', 3046),
 ('😘', 3015),
 ('🏽', 2822),
 ('🍔', 2757),
 ('♂', 2655),
 ('🔥', 2534),
 ('💦', 2378),
 ('🍾', 2345),
 ('🍜', 2283),
 ('🍴', 2158),
 ('🍳', 2036),
 ('🔌', 1962),
 ('🍗', 1945),
 ('🚕', 1903),
 ('👶', 1874),
 ('💕', 1869),
 ('💃', 1784),
 ('🍣', 1776),
 ('🍟', 1761),
 ('🥂', 1750),
 ('💵', 1726),
 ('💡', 1663),
 ('🏡', 1657),
 ('🐶', 1655),
 ('⚡', 1648),
 ('⛳', 1606),
 ('🍆', 1575),
 ('🍸', 1524),
 ('🎁', 1420),
 ('💇', 1372),
 ('🎂', 1362),
 ('🌯', 1326),
 ('⚾', 1295),
 ('✈', 1201),
 ('🚘', 1190)]

In [87]:
def get_text_from_emojis(emojis):
    emoji_text = [emoji.demojize(f'{em}') for em in emojis]
    return emoji_text

In [88]:
emoji_text = get_text_from_emojis(unique_emojis)

In [89]:
emoji_text

[':beating_heart:',
 ':five-thirty:',
 ':smiling_face_with_horns:',
 ':BACK_arrow:',
 ':station:',
 ':breast-feeding:',
 ':bullet_train:',
 ':full_moon:',
 ':no_one_under_eighteen:',
 ':clapping_hands:',
 ':graduation_cap:',
 ':weary_cat_face:',
 ':fox_face:',
 ':sun:',
 ':lemon:',
 ':pretzel:',
 ':trident_emblem:',
 ':old_key:',
 ':kitchen_knife:',
 ':slot_machine:',
 ':kick_scooter:',
 ':right_arrow:',
 ':person_tipping_hand:',
 ':face_with_monocle:',
 ':flashlight:',
 ':white_flower:',
 ':vibration_mode:',
 ':joystick:',
 ':shallow_pan_of_food:',
 ':eyes:',
 ':socks:',
 ':Japanese_dolls:',
 ':world_map:',
 ':regional_indicator_symbol_letter_g:',
 ':camel:',
 ':fish_cake_with_swirl:',
 ':crocodile:',
 ':love_letter:',
 ':spider:',
 ':passport_control:',
 ':Taurus:',
 ':backhand_index_pointing_right:',
 ':regional_indicator_symbol_letter_i:',
 ':ogre:',
 ':rolling_on_the_floor_laughing:',
 ':french_fries:',
 ':mouth:',
 ':chart_increasing_with_yen:',
 ':Japanese_congratulations_button